### 1.1 version of scrapper
+ proxy

In [1]:
# -*- coding: utf-8 -*-
from urllib2 import urlopen
from lxml import etree
import urllib
from lxml import html
from bs4 import BeautifulSoup
import io
import pandas as pd
from io import StringIO, BytesIO
import pandas as pd
import numpy as np
import re
from collections import deque
import json
import urllib2
import time

In [2]:
import pandas as pd
url = "https://www.avito.ru/moskva/tovary_dlya_kompyutera/komplektuyuschie/videokarty?p=2"
page = urllib.urlopen(url).read()
soup = BeautifulSoup(page, 'html.parser')

In [11]:
def save_ad_phone_image(ad_id, ad_image_hash):
        ad_image_file = open('%s' % ad_id, "wb")
        ad_image_file.write(ad_image_hash.decode('base64'))
        ad_image_file.close()
        
class PageParser:
    
    """
    example page: https://www.avito.ru/moskva/tovary_dlya_kompyutera/nvidia_gtx_1080_amd_radeon_r9_270x_i_drugie_modeli_728193995
    extract:
        phone
        text description
        adress
        name
        price
        how to use: 
        >> parser = PageParser(url)
        >> parser.get_user_name()
    """
    
    description = None
    price = None
    def __init__(self, url, proxies, ip_cnt):
        self.url = url #!!!!
        self.page = urllib2.urlopen(self.url).read()
        self.soup = BeautifulSoup(self.page, 'html.parser')
        if proxies == None: 
            proxies = []
        self.proxies = proxies
        self.ip_usage = ip_cnt
    def urlopen(self, url):
        # open new ip each time using counter ip_usage
        prx = self.ip_usage % (len(self.proxies) + 1)
        self.ip_usage = (self.ip_usage + 1) % (len(self.proxies) + 1)
        if (prx == 0):
            print("usual ip")
            test_proxy = urllib2.ProxyHandler({})
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            return urllib2.urlopen(url)
        else:
            prx -= 1
            print self.proxies[prx]
            test_proxy = urllib2.ProxyHandler(self.proxies[prx])
            opener = urllib2.build_opener(test_proxy)
            urllib2.install_opener(opener)
            answer = urllib2.urlopen(url)
            return answer
    def find_price(self):
        return self.get_variable('avito.item.price')
    def get_title(self):
        tag_ = self.soup.find('div', class_="sticky-header-prop")
        return tag_.text.strip()
    def find_text_description(self):
        tag_ = self.soup.find('meta', attrs={"property" : "og:description"})
        self.description = tag_["content"]
        return self.description
    def find_id_number(self):
        tag_ = self.soup.find('div', class_='title-info-metadata-item')
        self.id_number = tag_.text
        self.id_number = self.get_number_from_str(self.id_number)
        return self.id_number
    def get_number_from_str(self, s):
        # переписать нормально мб
        while (not re.match('[0-9]', s[0])):
            s = s[1:]
        ans = ""
        i = 0
        while (re.match('[0-9]', s[i:])):
            i += 1
        return s[:i]
    # get image url by pkey and ID
    def get_image_pkey(self, ad_id=False, ad_phone=False):
        if ad_id and ad_phone:
            ad_subhash = re.findall(r'[0-9a-f]+', ad_phone)
            if int(ad_id) % 2 == 0:
                ad_subhash.reverse()
            ad_subhash = ''.join(ad_subhash)
            return ad_subhash[::3]
    def get_phone_json(self, id_, pkey_, sHash_, url_):
        req = "https://www.avito.ru/items/phone/{0}?pkey={1}&vsrc=r&searchHash={2}".format(id_, pkey_, sHash_)
        ref = 'https://www.avito.ru' + url_
        request = urllib2.Request(req, headers={'Referer': ref})
        response = self.urlopen(request)
        html = response.read()
        return html
    def get_variable(self, s):
        pre = re.compile(s + " = '(.*)';")    
        scripts = self.soup.find_all('script')
        answer = ""
        for i in scripts:
            try:
                ans = re.findall(pre, i.string)
                if len(ans) != 0:
                    answer = ans[0].strip()
            except:
                pass
        return answer   
    def get_ad_phone_hash(self):
        return self.get_variable('avito.item.phone')
    
    def get_avito_item_searchHash(self):
        return self.get_variable('avito.item.searchHash')
    
    def get_avito_item_id(self):
        return self.get_variable('avito.item.id')
    
    def get_user_name(self):
        # <div class="seller-info-name">
        tag_ = self.soup.find('div', class_='seller-info-name')
        return tag_.text.strip()
    
    def get_user_adress(self):
        tags = self.soup.find_all('div', class_='seller-info-prop')
        for i in tags:
            tg = i.find('div', class_='seller-info-label')
            if (tg.text == u'Адрес'):
                return i.find('div', class_='seller-info-value').text.strip()
        return "NONE"
    
    def get_phone_image(self):
        id_ = str(self.get_avito_item_id())
        phone_ = self.get_ad_phone_hash()
        pkey_ = self.get_image_pkey(id_, phone_)
        url_ = self.get_variable('avito.item.url')
        json = self.get_phone_json(id_, pkey_, self.get_avito_item_searchHash(), url_)
        self.json = json
        return self.json
    
    def save_image(self):
        ss = json.loads(self.get_phone_image())['image64']
        image = ss[ss.find(','):]
        save_ad_phone_image(parser.get_avito_item_id(), image)
    ################################
    #       mobile version         #
    ################################
    def get_phone_mobile(self, Hash_, url_):
        req = 'https://m.avito.ru' + url_ + '?async&searchHash=' + Hash_
        ref = 'https://m.avito.ru' + url_
        request = urllib2.Request(req, headers={'Referer': ref})
        response = self.urlopen(request)
        html = response.read()
        return html
    def get_phone_from_mobile(self):
        hash_ = self.get_avito_item_searchHash()
        """class="person-action button button-solid button-blue button-large action-link link action-show-number
        js-action-show-number  "
        href="/moskva/predlozheniya_uslug/arenda_avtovyshki_15m_801012276/phone/3f124991e8b0dbb97b7c025c46b1fe6f"
        """
        tag_ = self.soup.find('a', class_="action-show-number")
        url_ = tag_['href']
        json_ = self.get_phone_mobile(hash_, url_)
        return json.loads(json_)['phone']

In [12]:
class Data:
    def __init__(self, proxies=None):
        """
        create a deque which is trying to get-and-scrap pages by url from Avito
        extracts data from pages via PageParser and return DataFrame object
        NOTE blocked if using with same ip non-stop more then 10 times (10 links)
        """
        self.index = 0
        self.links = deque()
        self.descriptions = deque()
        self.prices = deque()
        self.mobiles = deque()
        self.user_names = deque()
        self.adresses = deque()
        self.titles = deque()
        self.proxies = proxies
        self.ip_cnt = 0
    def add_link(self, lnk):
        self.links.append(lnk)
    def parse_next(self):
        self.ip_cnt += 1
        # full
        parser = PageParser(self.links[self.index], proxies=self.proxies, ip_cnt=self.ip_cnt)
        self.descriptions.append(parser.find_text_description())
        self.prices.append(parser.find_price())
        self.user_names.append(parser.get_user_name())
        self.adresses.append(parser.get_user_adress())
        self.titles.append(parser.get_title())
        # mobile
        parser = PageParser(self.links[self.index].replace('www', 'm'), self.proxies, ip_cnt=self.ip_cnt)
        self.mobiles.append(parser.get_phone_from_mobile())
        self.index += 1
    def get_DataFrame(self):
        d = {'title' : self.titles, 'Price' : self.prices, 'Description' : self.descriptions, 'phone' : self.mobiles, 
            'name' : self.user_names, 'adress' : self.adresses}
        return pd.DataFrame(data=d)

In [13]:
data = Data()

Extract pages and data in data = Data()

In [15]:
url = "https://linkedin.com/in/vsopin"

In [16]:

proxies=[{'http':'http://5.8.36.174:8085'}]
test_proxy = urllib2.ProxyHandler(proxies[0])
opener = urllib2.build_opener(test_proxy)
urllib2.install_opener(opener)

soup = BeautifulSoup(urllib2.urlopen(url).read(), 'html.parser')
print soup.prettify()

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:590)>

In [27]:
autos = data.get_DataFrame()
autos

,Description,Price,adress,name,phone,title
0,Комплектация: Стандарт Комплектация: Аудиопод...,395900,Москва,АвтоГЕРМЕС LADA Каширка,8 495 663-18-94,"LADA Granta, 2017"
1,"все по телефону.САЛОНЫ И ПЕРЕКУПЫ НА,УЙ.есть б...",535000,"Москва, м. Нахимовский проспект, Нахимовский п...",Владимир,8 906 046-98-82,"LADA Largus, 2015"


In [28]:
autos.sort_values(by='Price')

,Description,Price,adress,name,phone,title
0,Комплектация: Стандарт Комплектация: Аудиопод...,395900,Москва,АвтоГЕРМЕС LADA Каширка,8 495 663-18-94,"LADA Granta, 2017"
1,"все по телефону.САЛОНЫ И ПЕРЕКУПЫ НА,УЙ.есть б...",535000,"Москва, м. Нахимовский проспект, Нахимовский п...",Владимир,8 906 046-98-82,"LADA Largus, 2015"
